In [61]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

In [62]:
meteostat_1 = pd.read_csv('processed/442400146024_daily_meteostat.csv')
meteostat_1

,READ_DATE,KWH_EXP,tavg,tmin,tmax,wdir,wspd,pres
0,2019-01-01,0.000000,25.7,24.8,30.2,44.0,6.4,1008.9
1,2019-01-02,0.076469,26.0,24.1,29.2,21.0,5.4,1009.2
2,2019-01-03,0.310542,24.6,24.0,27.3,340.0,4.1,1010.3
3,2019-01-04,0.066232,26.2,23.0,29.8,340.0,6.9,1011.4
4,2019-01-05,1.070875,26.9,23.6,30.2,284.0,6.3,1011.7
...,...,...,...,...,...,...,...,...
1973,2024-05-27,39.352355,28.3,24.4,34.0,126.0,9.9,1009.9
1974,2024-05-28,36.329893,28.2,24.1,33.9,125.0,8.0,1009.4
1975,2024-05-29,43.653419,29.0,24.2,33.0,130.0,7.7,1009.1
1976,2024-05-30,40.912175,29.7,25.0,35.0,106.0,7.8,1009.7


In [63]:
df_weather = pd.read_csv('processed/weather_2019_2024_standard.csv')
df_weather

,Tanggal,Tavg,RH_avg,RR,ss
0,2019-01-01,25.7,96.0,12.9,7.5
1,2019-01-02,25.9,90.0,42.7,6.7
2,2019-01-03,24.9,94.0,0.5,0.1
3,2019-01-04,25.8,88.0,34.6,0.1
4,2019-01-05,26.7,88.0,2.1,3.2
...,...,...,...,...,...
2003,2024-06-26,27.4,76.0,0.0,8.0
2004,2024-06-27,27.8,77.0,0.0,9.9
2005,2024-06-28,28.4,76.0,0.0,10.8
2006,2024-06-29,28.6,65.0,0.0,10.0


In [64]:
meteostat_1['READ_DATE'] = pd.to_datetime(meteostat_1['READ_DATE'])
df_weather['Tanggal'] = pd.to_datetime(df_weather['Tanggal'])

meteostat_1['READ_DATE_day'] = meteostat_1['READ_DATE'].dt.date
df_weather['Tanggal'] = df_weather['Tanggal'].dt.date

meteostat_1 = pd.merge(meteostat_1, df_weather, how='left', left_on='READ_DATE_day', right_on='Tanggal')
meteostat_1.drop(columns=['Tanggal'], inplace=True)

meteostat_1.set_index('READ_DATE', inplace=True)
meteostat_1.drop(columns=['READ_DATE_day'], inplace=True)

In [65]:
meteostat_1

,KWH_EXP,tavg,tmin,tmax,wdir,wspd,pres,Tavg,RH_avg,RR,ss
READ_DATE,,,,,,,,,,,
2019-01-01,0.000000,25.7,24.8,30.2,44.0,6.4,1008.9,25.7,96.0,12.9,7.5
2019-01-02,0.076469,26.0,24.1,29.2,21.0,5.4,1009.2,25.9,90.0,42.7,6.7
2019-01-03,0.310542,24.6,24.0,27.3,340.0,4.1,1010.3,24.9,94.0,0.5,0.1
2019-01-04,0.066232,26.2,23.0,29.8,340.0,6.9,1011.4,25.8,88.0,34.6,0.1
2019-01-05,1.070875,26.9,23.6,30.2,284.0,6.3,1011.7,26.7,88.0,2.1,3.2
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-27,39.352355,28.3,24.4,34.0,126.0,9.9,1009.9,28.7,73.0,0.0,10.4
2024-05-28,36.329893,28.2,24.1,33.9,125.0,8.0,1009.4,28.2,75.0,0.0,8.2
2024-05-29,43.653419,29.0,24.2,33.0,130.0,7.7,1009.1,29.0,77.0,0.0,10.7


In [66]:
result = seasonal_decompose(meteostat_1['KWH_EXP'], model='additive', period=7)

meteostat_1['trend'] = result.trend
meteostat_1['seasonal'] = result.seasonal
meteostat_1['residual'] = result.resid

In [67]:
meteostat_1.isna().sum()

KWH_EXP     0
tavg        0
tmin        0
tmax        0
wdir        0
wspd        0
pres        0
Tavg        0
RH_avg      0
RR          0
ss          0
trend       6
seasonal    0
residual    6
dtype: int64

In [68]:
meteostat_1['trend'].fillna(meteostat_1['trend'].mean(), inplace=True)
meteostat_1['residual'].fillna(meteostat_1['residual'].mean(), inplace=True)
meteostat_1.fillna(0, inplace=True)

/tmp/ipykernel_4566/1491972629.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  meteostat_1['trend'].fillna(meteostat_1['trend'].mean(), inplace=True)
/tmp/ipykernel_4566/1491972629.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value

In [69]:
meteostat_1

,KWH_EXP,tavg,tmin,tmax,wdir,wspd,pres,Tavg,RH_avg,RR,ss,trend,seasonal,residual
READ_DATE,,,,,,,,,,,,,,
2019-01-01,0.000000,25.7,24.8,30.2,44.0,6.4,1008.9,25.7,96.0,12.9,7.5,45.079645,-0.441251,-0.001857
2019-01-02,0.076469,26.0,24.1,29.2,21.0,5.4,1009.2,25.9,90.0,42.7,6.7,45.079645,-0.103000,-0.001857
2019-01-03,0.310542,24.6,24.0,27.3,340.0,4.1,1010.3,24.9,94.0,0.5,0.1,45.079645,-0.120766,-0.001857
2019-01-04,0.066232,26.2,23.0,29.8,340.0,6.9,1011.4,25.8,88.0,34.6,0.1,0.603440,0.298958,-0.836166
2019-01-05,1.070875,26.9,23.6,30.2,284.0,6.3,1011.7,26.7,88.0,2.1,3.2,0.626695,0.308076,0.136104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-27,39.352355,28.3,24.4,34.0,126.0,9.9,1009.9,28.7,73.0,0.0,10.4,37.831261,-0.188571,1.709666
2024-05-28,36.329893,28.2,24.1,33.9,125.0,8.0,1009.4,28.2,75.0,0.0,8.2,38.348749,-0.441251,-1.577605
2024-05-29,43.653419,29.0,24.2,33.0,130.0,7.7,1009.1,29.0,77.0,0.0,10.7,45.079645,-0.103000,-0.001857


In [70]:
# save to csv
meteostat_1.to_csv('processed/a_442400146024_engineered.csv', index=False)